In [92]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
        print(os.path.join(dirname, filename))


/kaggle/input/sbki-mkb/Dataset (1).csv


# LEVEL 1




In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("/kaggle/input/sbki-mkb/Dataset (1).csv")

numeric_df = df.select_dtypes(include=[np.number])

corr_matrix = numeric_df.corr()

plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, 
            annot=True, 
            fmt=".2f", 
            cmap="coolwarm", 
            linewidths=0.5, 
            square=True, 
            cbar_kws={"shrink": .75})

plt.title("Correlation Heatmap of Numeric Features", fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
df[['Feature_1', 'Feature_2', 'Feature_3']].hist(bins=15, figsize=(12, 6), layout=(1, 3))
plt.suptitle("Histograms of Anonymized Features")
plt.show()


In [ ]:

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for i, feature in enumerate(['Feature_1', 'Feature_2', 'Feature_3']):
    sns.scatterplot(x=df[feature], y=df['G3'], ax=axes[i])
    axes[i].set_title(f'{feature} vs G3')
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 4))
for i, feature in enumerate(['Feature_1', 'Feature_2', 'Feature_3']):
    plt.subplot(1, 3, i+1)
    sns.boxplot(data=df, x='sex', y=feature)
    plt.title(f"{feature} by Sex")
plt.tight_layout()
plt.show()


# LEVEL 2

In [ ]:

missing_info = df.isnull().sum()
missing_info = missing_info[missing_info > 0].sort_values(ascending=False)

print("Missing Values in Each Column:")
print(missing_info)


In [ ]:
df['higher'] = df['higher'].fillna(df['higher'].mode()[0])
df['famsize'] = df['famsize'].fillna(df['famsize'].mode()[0])

df['Fedu'] = df['Fedu'].fillna(df['Fedu'].median())
df['traveltime'] = df['traveltime'].fillna(df['traveltime'].median())
df['freetime'] = df['freetime'].fillna(df['freetime'].median())

for col in ['absences', 'Feature_1', 'Feature_2', 'Feature_3', 'G2']:
    df[col] = df[col].fillna(df[col].median())


In [ ]:

print(df.isnull().sum().sort_values(ascending=False).head(10))


# LEVEL 3

**Do female and male students differ in their final grades?**

In [ ]:

plt.figure(figsize=(8, 5))
sns.violinplot(data=df, x='sex', y='G3', inner='box', palette='Set2')
plt.title('Distribution of Final Grades (G3) by Gender')
plt.xlabel('Gender')
plt.ylabel('Final Grade (G3)')
plt.show()


**How does mother's education level(medu) impact student performance?**

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.barplot(data=df, x='Medu', y='G3', palette='Blues_d', errorbar=None)
plt.title("Average Final Grade (G3) by Mother's Education Level", fontsize=14)
plt.xlabel("Mother's Education Level (0 = none to 4 = higher ed)", fontsize=12)
plt.ylabel("Average Final Grade (G3)", fontsize=12)
plt.tight_layout()
plt.show()
plt.figure(figsize=(8, 6))
sns.barplot(data=df, x='Fedu', y='G3', palette='Greens_d', errorbar=None)
plt.title("Average Final Grade (G3) by Father's Education Level", fontsize=14)
plt.xlabel("Father's Education Level (0 = none to 4 = higher ed)", fontsize=12)
plt.ylabel("Average Final Grade (G3)", fontsize=12)
plt.tight_layout()
plt.show()


**Does alcohol consumption (Dalc) impact academic performance?**

In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(data=df, x='Dalc', y='G3', ci=None, palette='viridis')
plt.title('Average Final Grade (G3) by Daily Alcohol Consumption')
plt.xlabel('Daily Alcohol Consumption (Dalc)')
plt.ylabel('Average Final Grade (G3)')
plt.show()


In [ ]:

jitter_strength = 0.15
x_jitter = df['freetime'] + np.random.uniform(-jitter_strength, jitter_strength, size=len(df))
y_jitter = df['goout'] + np.random.uniform(-jitter_strength, jitter_strength, size=len(df))

plt.figure(figsize=(8,6))
plt.scatter(x_jitter, y_jitter, alpha=0.6, edgecolors='w', s=80)
plt.title('Scatterplot of Free Time vs Going Out with Friends (with Jitter)')
plt.xlabel('Free Time (after school)')
plt.ylabel('Going Out with Friends (frequency)')
plt.xticks([1,2,3,4,5])
plt.yticks([1,2,3,4,5])
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()


**Does access to the internet at home affect absences?**

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(data=df, x='internet', y='absences', palette='pastel')
plt.title('Absences by Internet Access at Home')
plt.xlabel('Internet Access at Home')
plt.ylabel('Number of Absences')
plt.show()


# LEVEL 4

**ENSEMBLE VOTING**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
df['romantic'] = LabelEncoder().fit_transform(df['romantic'])
X = df.drop(columns=['romantic'])
y = df['romantic']
X = pd.get_dummies(X, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
log_clf = LogisticRegression(max_iter=1000)
rf_clf = RandomForestClassifier(random_state=42)
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
ensemble_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rf_clf), ('xgb', xgb_clf)],
    voting='soft'
)
ensemble_clf.fit(X_train, y_train)
y_pred = ensemble_clf.predict(X_test)
print("Ensemble Model Performance:\n")
print(classification_report(y_test, y_pred))


**XGBoost**

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
df['romantic'] = LabelEncoder().fit_transform(df['romantic'])  # Yes → 1, No → 0
X = df.drop(columns=['romantic'])
y = df['romantic']
X = pd.get_dummies(X, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xgb_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3
)

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
print("XGBoost Performance:\n")
print(classification_report(y_test, y_pred))


**CATBOOST**

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
X = df.drop(columns=['romantic'])
y = df['romantic']
cat_features = [i for i, col in enumerate(X.columns) if X[col].dtype == 'object']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.1,
    depth=6,
    eval_metric='F1',
    random_seed=42,
    verbose=50
)

model.fit(X_train, y_train, cat_features=cat_features)
y_pred = model.predict(X_test)

print("CatBoost Performance:\n")
print(classification_report(y_test, y_pred))


# LEVEL 5

In [ ]:
pip uninstall -y scikit-learn shap


In [ ]:
pip install scikit-learn==1.2.2 shap==0.42.1


In [ ]:

import sklearn
import shap
import sys
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib


print("Pandas version:", pd.__version__)
print("Seaborn version:", sns.__version__)
print("Matplotlib version:", matplotlib.__version__)


print("Python version:", sys.version)
print("NumPy version:", np.__version__)


print("sklearn version:",sklearn.__version__)
print("shap version:",shap.__version__)


In [ ]:

features = ['goout', 'Dalc']
X_two = df[features]
y_two = df['romantic'].astype(int)

X_train_2D, X_test_2D, y_train_2D, y_test_2D = train_test_split(X_two, y_two, test_size=0.2, random_state=42)
from catboost import CatBoostClassifier
model_2D = CatBoostClassifier(iterations=300, learning_rate=0.01, depth=4, verbose=0)
model_2D.fit(X_train_2D, y_train_2D)
import matplotlib.pyplot as plt
import numpy as np

def plot_decision_boundary_catboost(model, X, y, features):
    h = 0.1
    x_min, x_max = X[features[0]].min() - 1, X[features[0]].max() + 1
    y_min, y_max = X[features[1]].min() - 1, X[features[1]].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    grid = pd.DataFrame({features[0]: xx.ravel(), features[1]: yy.ravel()})
    Z = model.predict(grid)
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, alpha=0.4, cmap=plt.cm.RdYlBu)
    plt.scatter(X[features[0]], X[features[1]], c=y, cmap=plt.cm.RdYlBu, edgecolors='k')
    plt.xlabel(features[0])
    plt.ylabel(features[1])
    plt.title("CatBoost Decision Boundary")
    plt.show()

plot_decision_boundary_catboost(model_2D, X_test_2D, y_test_2D, features)


In [ ]:
import shap
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test, plot_type="bar")


In [ ]:
import shap
X_test_copy = X_test.copy()
X_test_copy['pred'] = model.predict(X_test)
yes_index = X_test_copy[X_test_copy['pred'] == 1].index[0]
no_index = X_test_copy[X_test_copy['pred'] == 0].index[0]
yes_pos = X_test.index.get_loc(yes_index)
no_pos = X_test.index.get_loc(no_index)
student_yes = X_test.iloc[yes_pos]
student_no = X_test.iloc[no_pos]
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[yes_pos], student_yes)


In [ ]:
import shap
X_test_copy = X_test.copy()
X_test_copy['pred'] = model.predict(X_test)
yes_index = X_test_copy[X_test_copy['pred'] == 1].index[0]
no_index = X_test_copy[X_test_copy['pred'] == 0].index[0]
yes_pos = X_test.index.get_loc(yes_index)
no_pos = X_test.index.get_loc(no_index)
student_yes = X_test.iloc[yes_pos]
student_no = X_test.iloc[no_pos]
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[no_pos], student_no)
